In [1]:
import numpy as np
import pandas as pd

In [36]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [37]:
movies = movies.merge(credits,on='title')

In [35]:
movies.info()

<class 'pandas.core.series.Series'>
Index: 4806 entries, 0 to 4808
Series name: genres
Non-Null Count  Dtype 
--------------  ----- 
4806 non-null   object
dtypes: object(1)
memory usage: 204.1+ KB


In [38]:
# genres , id , title , keywords , overview , cast , crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [12]:
# check if any colummn has null values

movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [39]:
movies.dropna(inplace=True)

In [14]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [15]:
movies.duplicated().sum()

0

In [40]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [41]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# ['Action','Adventure','Fantasy','SciFi']

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [42]:
movies['genres'] = movies['genres'].apply(convert)

In [44]:
movies['keywords'] = movies['keywords'].apply(convert)

In [47]:
def convert3(obj):
    L = []
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
           L.append(i['name'])
           counter += 1
        else:
            break
    return L


In [50]:
movies['cast']=movies['cast'].apply(convert3)

In [52]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break;
    return L


In [54]:
movies['crew']=movies['crew'].apply(fetch_director)

In [56]:
movies['overview']=  movies['overview'].apply(lambda x:x.split())

In [61]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']= movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']= movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [66]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [68]:
new_df = movies[['movie_id','title','tags']]

In [73]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\DELL\AppData\Local\Temp\ipykernel_10884\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [77]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\DELL\AppData\Local\Temp\ipykernel_10884\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [79]:
import nltk

In [81]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [85]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)                    

In [87]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10884\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [93]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [94]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

In [102]:
similarity = cosine_similarity(vectors)

In [105]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key= lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [106]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [108]:
import pickle


In [109]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [110]:
pickle.dump(similarity,open('similarity.pkl','wb'))